In [3]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset
!unzip -q stanford-dogs-dataset.zip -d stanford-dogs-dataset

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset
License(s): other
100% 748M/750M [00:32<00:00, 23.2MB/s]
100% 750M/750M [00:32<00:00, 24.4MB/s]


In [4]:
!ls stanford-dogs-dataset

annotations  images


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import glob
import xml.etree.ElementTree as ET
from PIL import Image

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
dataset_path = 'stanford-dogs-dataset'
images_path = os.path.join(dataset_path, "images")
annotations_path = os.path.join(dataset_path, "annotation")

In [8]:
import os
import shutil
import random

# Define paths
dataset_dir = 'stanford-dogs-dataset/images/Images'
annotations_dir = 'stanford-dogs-dataset/annotations/Annotation'
filtered_dir = 'filtered-dogs'
filtered_annotations_dir = os.path.join(filtered_dir, 'Annotation')
filtered_images_dir = os.path.join(filtered_dir, 'Images')

# Ensure the filtered directory is clean
if os.path.exists(filtered_dir):
    shutil.rmtree(filtered_dir)
os.makedirs(filtered_annotations_dir)
os.makedirs(filtered_images_dir)

# List all species
all_species = os.listdir(dataset_dir)
print(f'Total species: {len(all_species)}')

# Randomly select 10 species
selected_species = random.sample(all_species, 10)
print(f'Selected species: {selected_species}')

# Copy selected species to the filtered directory
for species in selected_species:
    species_image_path = os.path.join(dataset_dir, species)
    species_annotation_path = os.path.join(annotations_dir, species)
    target_image_path = os.path.join(filtered_images_dir, species)
    target_annotation_path = os.path.join(filtered_annotations_dir, species)

    # Copy image files
    shutil.copytree(species_image_path, target_image_path)

    # Copy annotation files
    shutil.copytree(species_annotation_path, target_annotation_path)

print("Selected species images and annotations have been copied to the filtered directory.")

Total species: 120
Selected species: ['n02086646-Blenheim_spaniel', 'n02112137-chow', 'n02098413-Lhasa', 'n02092339-Weimaraner', 'n02093256-Staffordshire_bullterrier', 'n02096051-Airedale', 'n02087394-Rhodesian_ridgeback', 'n02090721-Irish_wolfhound', 'n02099712-Labrador_retriever', 'n02091134-whippet']
Selected species images and annotations have been copied to the filtered directory.


In [9]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.0],
    channel_shift_range=0.2
)

In [10]:
images_dir = filtered_images_dir

In [11]:
train_generator = datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

Found 1471 images belonging to 10 classes.


In [12]:
validation_generator = datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

Found 364 images belonging to 10 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 1835 images belonging to 10 classes.


In [14]:
print(f'Total training samples: {train_generator.samples}')
print(f'Total validation samples: {validation_generator.samples}')
print(f'Total test samples: {test_generator.samples}')

Total training samples: 1471
Total validation samples: 364
Total test samples: 1835


In [15]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [17]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [19]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[early_stopping]
)

Epoch 1/10
22/22 [==============================] - 227s 10s/step - loss: 4.4095 - accuracy: 0.0945 - val_loss: 3.9504 - val_accuracy: 0.1000
Epoch 2/10
22/22 [==============================] - 233s 11s/step - loss: 3.6279 - accuracy: 0.1144 - val_loss: 3.3190 - val_accuracy: 0.1375
Epoch 3/10
22/22 [==============================] - 224s 10s/step - loss: 3.1121 - accuracy: 0.1130 - val_loss: 2.9168 - val_accuracy: 0.1250
Epoch 4/10
22/22 [==============================] - 243s 11s/step - loss: 2.7946 - accuracy: 0.1237 - val_loss: 2.6777 - val_accuracy: 0.1250
Epoch 5/10
22/22 [==============================] - 226s 10s/step - loss: 2.6004 - accuracy: 0.1251 - val_loss: 2.5264 - val_accuracy: 0.1375
Epoch 6/10
22/22 [==============================] - 223s 10s/step - loss: 2.4744 - accuracy: 0.1500 - val_loss: 2.4372 - val_accuracy: 0.1844
Epoch 7/10
22/22 [==============================] - 231s 10s/step - loss: 2.4092 - accuracy: 0.1485 - val_loss: 2.3698 - val_accuracy: 0.1781
Epoch 

In [20]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy}')

29/29 [==============================] - 77s 3s/step - loss: 2.2855 - accuracy: 0.1777
Test accuracy: 0.17765668034553528
